In [1]:
import torch.nn as nn
import torch
import torch.nn.utils.rnn as rnn
import statistics
import nltk # 없으시면 설치하세요: pip install nltk
import random
import collections
import time
import numpy as np # optinal

In [2]:
# 특정 GPU를 사용할 경우
# GPU_NUM=1 # 1번 gpu를 사용할 경우.
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU
# print ('Current cuda device ', torch.cuda.current_device()) # check

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
## Dictionary class 선언
class Dictionary(object):
    def __init__(self, dataset, size):
        ## init vocab ##
        self.word2idx = {'<pad>':0, '<sos>': 1, '<eos>': 2, '<unk>': 3} # 사전 
        self.idx2word = ['<pad>', '<sos>', '<eos>', '<unk>'] # inverted diction

        self.build_dict(dataset, size)

    def __call__(self, word):
        return self.word2idx.get(word, self.word2idx['<unk>']) # if word does not exist in vocab then return unk idx

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def build_dict(self, dataset, dict_size):
        """Tokenize a text file."""
        total_words = (word for sent in dataset for word in sent) # store all words into tuple
        word_freq = collections.Counter(total_words) # count the number of each word: ex) ('The': 10000, 'a': 5555, ...)
        vocab = sorted(word_freq.keys(), key=lambda word: (-word_freq[word], word)) # sort by frequency
        vocab = vocab[:dict_size]
        for word in vocab:
            self.add_word(word)

    def __len__(self):
        return len(self.idx2word)


In [4]:
## Brown dataset Preprocessing (NLTK)
def brown_dataset(min=5, max=30):
    nltk.download('brown')

    # get sentences with the length between min and max
    # convert all words into lower-case
    all_seq = [[token.lower() for token in seq] for seq in nltk.corpus.brown.sents() 
               if min <= len(seq) <= max]

    random.shuffle(all_seq) # shuffle
    return all_seq

In [5]:
## Download Brown dataset
dataset = brown_dataset()
print(len(dataset))
## print some part
print(dataset[0])
print(dataset[1])
print(dataset[2])

[nltk_data] Downloading package brown to /home/piai/nltk_data...
[nltk_data]   Package brown is already up-to-date!


43450
['du', 'pont', ',', 'christiana', ',', 'and', 'delaware', 'were', 'to', 'be', 'enjoined', 'from', 'acquiring', 'stock', 'in', 'or', 'exercising', 'control', 'over', 'general', 'motors', ';', ';']
['troubled', ',', 'he', 'continued', 'along', 'the', 'corridor', ',', 'poking', 'his', 'head', 'into', 'the', 'next', 'office', 'for', 'a', 'careful', 'look', 'around', '.']
['another', 'remained', 'when', 'an', 'american', 'army', 'car', 'was', 'recovered', 'but', 'with', 'a', 'broken', 'glass', '.']


In [6]:
## Data handler class 선언
class Corpus(object):
    def __init__(self, dataset, device, dict_size=20000, train_ratio=0.97):
        train_size = int(len(dataset) * train_ratio)
        self.device = device
        self.dictionary = Dictionary(dataset, dict_size)
        self.train = dataset[:train_size] # [0 ~ train_size]
        self.valid = dataset[train_size:] # [train_size: len(dataset)]

    def indexing(self, dat):
        # dat = list(list)
        src_idxes = [] # 모델 입력
        tgt_idxes = [] # 모델 정답
        for sent in dat:
            src_idx = [self.dictionary('<sos>')] + [self.dictionary(word) for word in sent]
            tgt_idx = [self.dictionary(word) for word in sent] + [self.dictionary('<eos>')]
            src_idxes.append(torch.tensor(src_idx).type(torch.int64))
            tgt_idxes.append(torch.tensor(tgt_idx).type(torch.int64))

        src_idxes = rnn.pad_sequence(src_idxes, batch_first=True).to(self.device) # shape = [B, L]
        tgt_idxes = rnn.pad_sequence(tgt_idxes, batch_first=True).to(self.device).view(-1) # flatten shape = [B * L]

        return src_idxes, tgt_idxes

    def batch_iter(self, batch_size, isTrain=True):
        dat = self.train if isTrain else self.valid
        if isTrain:
            random.shuffle(dat)

        for i in range(len(dat) // batch_size):
            batch = dat[i * batch_size: (i+1) * batch_size]
            src, tgt = self.indexing(batch)
            yield {'src': src, 'tgt': tgt}

In [7]:
corpus = Corpus(dataset, device)

In [8]:
# Dictionary 확인

for i, (key, val) in enumerate(corpus.dictionary.word2idx.items()):
    print('word:  {:10s} | index: {:5d} '.format(key, val))
    if i == 20:
        break


word:  <pad>      | index:     0 
word:  <sos>      | index:     1 
word:  <eos>      | index:     2 
word:  <unk>      | index:     3 
word:  the        | index:     4 
word:  .          | index:     5 
word:  ,          | index:     6 
word:  of         | index:     7 
word:  and        | index:     8 
word:  to         | index:     9 
word:  a          | index:    10 
word:  in         | index:    11 
word:  was        | index:    12 
word:  he         | index:    13 
word:  is         | index:    14 
word:  ''         | index:    15 
word:  ``         | index:    16 
word:  it         | index:    17 
word:  that       | index:    18 
word:  for        | index:    19 
word:  ;          | index:    20 


In [9]:
## indexing 함수 결과 확인

# case : 단일 문장 입력 시. 
sent = [dataset[1]]
idx_src, idx_tgt = corpus.indexing(sent)

print(sent)
print(idx_src) # <SOS> index로 시작
print(idx_tgt) # <EOS> index로 종료

print('-' * 90)
## case : 복수 문장 입력 시 (batching)
batch = [dataset[0], dataset[1]]
idx_src, idx_tgt = corpus.indexing(batch)

print(batch)
print(idx_src) # 가장 길이가 긴 문장 (dataset[0]) 보다 짧은 문장 (dataset[1]) 의 경우 남는 길이만큼 padding=0 삽입 확인.
print(idx_tgt)

[['troubled', ',', 'he', 'continued', 'along', 'the', 'corridor', ',', 'poking', 'his', 'head', 'into', 'the', 'next', 'office', 'for', 'a', 'careful', 'look', 'around', '.']]
tensor([[    1,  3860,     6,    13,   801,   269,     4,  3926,     6, 19310,
            21,   200,    69,     4,   218,   406,    19,    10,  1465,   216,
           155,     5]], device='cuda:0')
tensor([ 3860,     6,    13,   801,   269,     4,  3926,     6, 19310,    21,
          200,    69,     4,   218,   406,    19,    10,  1465,   216,   155,
            5,     2], device='cuda:0')
------------------------------------------------------------------------------------------
[['du', 'pont', ',', 'christiana', ',', 'and', 'delaware', 'were', 'to', 'be', 'enjoined', 'from', 'acquiring', 'stock', 'in', 'or', 'exercising', 'control', 'over', 'general', 'motors', ';', ';'], ['troubled', ',', 'he', 'continued', 'along', 'the', 'corridor', ',', 'poking', 'his', 'head', 'into', 'the', 'next', 'office', 'for', 'a',

In [10]:
## RNN Language model 선언

# Define network
class RNNModel(nn.Module):
    def __init__(self, ntoken, hidden_size, nlayers, dropout=0.1):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.embeddings = nn.Embedding(ntoken, hidden_size, padding_idx=0)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers, dropout=dropout, batch_first=True) # [B, L, Hidden_DIM]
        self.output_layer = nn.Linear(hidden_size, ntoken)
        self.sm = nn.LogSoftmax(dim=-1)

        self.ntoken = ntoken
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.output_layer.weight.data.uniform_(-initrange, initrange)
        self.output_layer.bias.data.zero_()

    def forward(self, input, hidden):
        emb = self.embeddings(input) # emb = (batch, length, dim)
        output, hidden = self.rnn(emb, hidden) # output = (batch. length. dim)
        output = self.drop(output)
        output = self.output_layer(output) # output = (batch, length, vocab_size)
        output = output.view(-1, self.ntoken) # output = (batch * length, vocab_size)

        return self.sm(output), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()) # to set init tensor with the same torch.dtype and torch.device
        return (weight.new_zeros(self.nlayers, bsz, self.hidden_size),
                weight.new_zeros(self.nlayers, bsz, self.hidden_size))


In [11]:
# Hyperparameters
batch_size = 60
hidden_size = 256
dropout = 0.2
max_epoch = 30

# build model
ntokens = len(corpus.dictionary)
model = RNNModel(ntokens, hidden_size, 1, dropout).to(device)
isTrain=True # Flag variable

# set loss func and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

/home/piai/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [12]:
##### Training / Evaluation Parts #######

In [13]:
# accuracy
def cal_acc(scores, target):
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

In [14]:
# train func.
def train():
    model.train() # Turn on training mode which enables dropout.
    mean_loss = []
    mean_acc = []
    start_time = time.time()

    for batch in corpus.batch_iter(batch_size):
        hidden = model.init_hidden(batch_size) # zero vectors for init hidden
        target = batch['tgt'] # flattened target 
        optimizer.zero_grad()
        output, hidden = model(batch['src'], hidden) # output = flatten output = [Batch_size * Length, vocab_size]

        # output shape = (batch * length, vocab_size)
        # target shape = (batch * length)   --> (batch * length, vocab_size) 로 one-hot distribtuion으로 내부적으로 변환되어 비교 수행
        loss = criterion(output, target) # compare between vocab_prob and answer_prob(one-hot converted)
        loss.backward()
        optimizer.step()

        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(output, target))

    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, total_time, mean_acc

In [15]:
# evaluation func.
def evaluate():
    model.eval() # Turn off dropout
    mean_loss = []
    mean_acc = []

    for batch in corpus.batch_iter(batch_size, isTrain=False):
        with torch.no_grad():
            hidden = model.init_hidden(batch_size)
            target = batch['tgt']
            output, hidden = model(batch['src'], hidden)
            loss = criterion(output, target)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(output, target))

    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, mean_acc

In [16]:
if isTrain: # set False if you don't need to train model
    start_time = time.time()

    for epoch in range(1, max_epoch+1):
        loss, epoch_time, accuracy = train()
        print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))

        if epoch % 10 == 0:
            loss, accuracy = evaluate()
            print('=' * 60)
            print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(loss, accuracy))
            print('=' * 60)

    with open('model.pt', 'wb') as f:
        print('save model at: ./model.pt')
        torch.save(model, f)

epoch    2 | times 18.493 |  loss: 5.689 | accuracy: 20.39
epoch    3 | times 18.316 |  loss: 5.065 | accuracy: 23.96
epoch    4 | times 18.352 |  loss: 4.714 | accuracy: 25.57
epoch    5 | times 18.390 |  loss: 4.399 | accuracy: 27.11
epoch    6 | times 14.636 |  loss: 4.114 | accuracy: 28.84
epoch    7 | times 13.448 |  loss: 3.863 | accuracy: 30.78
epoch    8 | times 15.717 |  loss: 3.655 | accuracy: 32.69
epoch    9 | times 18.421 |  loss: 3.485 | accuracy: 34.47
epoch   10 | times 18.484 |  loss: 3.345 | accuracy: 36.04
epoch   11 | times 18.027 |  loss: 3.228 | accuracy: 37.43
Evaluation | loss: 5.934 | accuracy: 23.76
epoch   12 | times 18.449 |  loss: 3.135 | accuracy: 38.61
epoch   13 | times 18.464 |  loss: 3.057 | accuracy: 39.55
epoch   14 | times 18.445 |  loss: 2.993 | accuracy: 40.35
epoch   15 | times 18.466 |  loss: 2.940 | accuracy: 41.00
epoch   16 | times 18.438 |  loss: 2.895 | accuracy: 41.61
epoch   17 | times 18.411 |  loss: 2.853 | accuracy: 42.11
epoch   18 | 

/home/piai/anaconda3/lib/python3.8/site-packages/torch/serialization.py:359: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "


In [22]:
def pred_sent_prob(sent):
    import numpy as np # optinal
    model.eval()
    sent_prob =0
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(sent)
        # 2. initial hidden 생성
        hidden = model.init_hidden(len(sent))
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src, hidden)
        # 4. 모델 확률분포로부터 정답 단어의 각 index에 대한 Log 확률 값 추출.
        for i in range(len(output)) :
        # 5. log 확률의 합.
            sent_prob += output[i][idx_tgt[i]]
        # 6. 결과 return
        return sent_prob

In [23]:
# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to(device)

    print('log prob of [the dog bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'dog', 'bark', '.']])))
    print('log prob of [the cat bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'cat', 'bark', '.']])))

    print('log prob of [boy am a i .]: {:3.3f}'.format(pred_sent_prob([['boy', 'am', 'a', 'i', '.']])))
    print('log prob of [i am a boy .]: {:3.3f}'.format(pred_sent_prob([['i', 'am', 'a', 'boy', '.']])))


load model from: ./model.pt
log prob of [the dog bark .]: -46.727
log prob of [the cat bark .]: -48.395
log prob of [boy am a i .]: -40.870
log prob of [i am a boy .]: -19.022


In [28]:
def pred_next_word(partial_sent, topN=3):
    import numpy as np # optinal
    model.eval()
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(partial_sent)
        # 2. initial hidden 생성
        hidden = model.init_hidden(len(partial_sent))
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src, hidden)
        # 4. topN에 해당하는 다음단어의 word index 추출: HINT) torch.topk() 활용
        values, idx_word = output.topk(topN)
        # 5. word index --> word 로 변환
        word_list = []
        for i in range(topN) :
            idx_category= idx_word[0][i].item()
            word_list.append(corpus.dictionary.idx2word[idx_category])
        # 6. topN word list 반환
        return word_list

In [29]:
partial_sent = [['the', 'next', 'word']]
N=3
candidates = pred_next_word(partial_sent, topN=N)

# print 
partial_sent = ' '.join(partial_sent[0])
print('Top {0} next words for a partial sentence [{1}] is: '.format(N, partial_sent))
print('===>', candidates)

Top 3 next words for a partial sentence [the next word] is: 
===> ['the', '``', 'he']
